<a href="https://colab.research.google.com/github/amgabatbayar/Uczenie-Maszynowe-w-Finansach---Marcin-Batbayar/blob/main/case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=9ff9856582cf97e02d55c7e696d4850feb8f58e2391217b24b3d36e99e3de072
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
!pip install yfinance

In [3]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [4]:
import datetime as dt
import yfinance as yf

AMZN = yf.download('AMZN', start='2024-01-01', end='2024-05-06')
AMZN.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-02,151.539993,152.380005,148.389999,149.929993,149.929993,47339400
2024-01-03,149.199997,151.050003,148.330002,148.470001,148.470001,49425500
2024-01-04,145.589996,147.380005,144.050003,144.570007,144.570007,56039800
2024-01-05,144.690002,146.589996,144.529999,145.240005,145.240005,45124800
2024-01-08,146.740005,149.399994,146.149994,149.100006,149.100006,46757100


In [5]:
class SmaCross_withSL(Strategy):
    n1 = 10
    n2 = 25

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
          price = self.data.Close
          long_sl = price * 0.99
          long_tp = price * 1.05
          if self.position.is_short:
            self.position.close()
            self.buy(size=0.5, sl=long_sl, tp=long_tp)
          elif self.position.is_long:
            pass
          else:
            self.buy(size=0.5, sl=long_sl, tp=long_tp)

        elif crossover(self.sma2, self.sma1):
          price = self.data.Close
          short_sl = price * 1.01
          short_tp = price * 0.95
          if(self.position.is_long):
            self.position.close()
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)
          elif self.position.is_short:
            pass
          else:
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)

In [7]:
bt = Backtest(AMZN, SmaCross_withSL,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    1.162791
Equity Final [$]                 10045.823949
Equity Peak [$]                  10045.823949
Return [%]                           0.458239
Buy & Hold Return [%]                24.19797
Return (Ann.) [%]                    1.348695
Volatility (Ann.) [%]                0.799623
Sharpe Ratio                         1.686664
Sortino Ratio                             inf
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       0.933111
Worst Trade [%]                      0.933111
Avg. Trade [%]                    

In [8]:
opt = bt.optimize(n1=[5,7,9,11,13,15], n2=[10,15, 20,25,30,35,40], maximize='Sharpe Ratio',
                  constraint=lambda p: p.n1 < p.n2)
print(opt)

print(opt._strategy)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: Run

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    2.325581
Equity Final [$]                 10267.303504
Equity Peak [$]                  10267.303504
Return [%]                           2.673035
Buy & Hold Return [%]                24.19797
Return (Ann.) [%]                    8.036353
Volatility (Ann.) [%]                3.739637
Sharpe Ratio                         2.148966
Sortino Ratio                             inf
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.512932
Worst Trade [%]                      5.512932
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
